In [76]:
import pulp as plp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [77]:
g_df = pd.DataFrame({'region' : ['W', 'E', 'W', 'E', 'W', 'E', 'W','W','W','W','E','W','E','E','E'],
                     'power_g' : [120, 50, 200, 400, 60, 50, 60, 100, 70, 50, 70, 45, 50, 60, 50],
                     'cost_g' : [0, 0, 15, 30, 32.5, 34, 36, 37.5, 39, 40, 60, 70, 100, 150, 200]})
d_df = pd.DataFrame({'region' : ['W','E','W','E','W','W','E','W','W','E','E','E'],
                     'power_d' : [250, 300, 120, 80, 40, 70, 60, 45, 30, 35, 25, 10],
                     'cost_d' : [200, 110, 100, 90, 85, 75, 65, 40, 38, 31, 24, 16]})
# East Region
power_g_e = g_df[g_df['region']=='E'].values[:,1].T.tolist()
cost_g_e = g_df[g_df['region']=='E'].values[:,2].T.tolist()
power_d_e = d_df[d_df['region']=='E'].values[:,1].T.tolist()
cost_d_e = d_df[d_df['region']=='E'].values[:,2].T.tolist()

# West Region
power_g_w = g_df[g_df['region']=='W'].values[:,1].T.tolist()
cost_g_w = g_df[g_df['region']=='W'].values[:,2].T.tolist()
power_d_w = d_df[d_df['region']=='W'].values[:,1].T.tolist()
cost_d_w = d_df[d_df['region']=='W'].values[:,2].T.tolist()

# Interconnection
limit = 40

In [78]:
ng_e = len(power_g_e)
nd_e = len(power_d_e)

ng_w = len(power_g_w)
nd_w = len(power_d_w)

set_ng_e = range(1, ng_e + 1)
set_nd_e = range(1, nd_e + 1)

set_ng_w = range(1, ng_w + 1)
set_nd_w = range(1, nd_w + 1)

cg_e = {(i): cost_g_e[i-1] for i in set_ng_e}
pg_e = {(i): power_g_e[i-1] for i in set_ng_e}
cd_e = {(i): cost_d_e[i-1] for i in set_nd_e}
pd_e = {(i): power_d_e[i-1] for i in set_nd_e}

cg_w = {(i): cost_g_w[i-1] for i in set_ng_w}
pg_w = {(i): power_g_w[i-1] for i in set_ng_w}
cd_w = {(i): cost_d_w[i-1] for i in set_nd_w}
pd_w = {(i): power_d_w[i-1] for i in set_nd_w}

In [79]:
opt_model = plp.LpProblem(name="Model")

In [80]:
yg_e  = {(i): plp.LpVariable(cat='Continuous', lowBound=0, upBound=pg_e[i], name="yg_e_{}".format(i)) for i in set_ng_e}
yd_e  = {(i): plp.LpVariable(cat='Continuous', lowBound=0, upBound=pd_e[i], name="yd_e_{}".format(i)) for i in set_nd_e}

yg_w  = {(i): plp.LpVariable(cat='Continuous', lowBound=0, upBound=pg_w[i], name="yg_w_{}".format(i)) for i in set_ng_w}
yd_w  = {(i): plp.LpVariable(cat='Continuous', lowBound=0, upBound=pd_w[i], name="yd_w_{}".format(i)) for i in set_nd_w}

theta = plp.LpVariable(cat='Continuous', lowBound=-limit, upBound=limit, name="theta")

In [81]:
constraint_1 = plp.LpConstraint(
                 e=plp.lpSum(yd_w[j] for j in set_nd_w) - plp.lpSum(yg_w[i] for i in set_ng_w)-theta,
                 sense=plp.LpConstraintEQ,
                 rhs=0,
                 name="constraint_eq_1")

constraint_2 = plp.LpConstraint(
                 e=plp.lpSum(yd_e[j] for j in set_nd_e) - plp.lpSum(yg_e[i] for i in set_ng_e)+theta,
                 sense=plp.LpConstraintEQ,
                 rhs=0,
                 name="constraint_eq_2")

In [83]:
constraint_2

1*theta + 1*yd_e_1 + 1*yd_e_2 + 1*yd_e_3 + 1*yd_e_4 + 1*yd_e_5 + 1*yd_e_6 + -1*yg_e_1 + -1*yg_e_2 + -1*yg_e_3 + -1*yg_e_4 + -1*yg_e_5 + -1*yg_e_6 + -1*yg_e_7 + 0 = 0

In [84]:
opt_model += constraint_1, "balance between supply and demand west"
opt_model += constraint_2, "balance between supply and demand east"

In [85]:
objective = plp.lpSum(yd_e[j]*cd_e[j] for j in set_nd_e) + plp.lpSum(yd_w[i]*cd_w[i] for i in set_nd_w) - (
            plp.lpSum(yg_e[j]*cg_e[j] for j in set_ng_e) + plp.lpSum(yg_w[i]*cg_w[i] for i in set_ng_w))

In [86]:
opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)

In [87]:
opt_model.solve()
#print(plp.LpStatus[opt_model.status])

1

In [88]:
opt_g_e_df = pd.DataFrame.from_dict(yg_e, orient="index", columns = ["variable_object"])
opt_g_w_df = pd.DataFrame.from_dict(yg_w, orient="index", columns = ["variable_object"])
opt_g_df = pd.concat([opt_g_e_df, opt_g_w_df], ignore_index = True)
opt_g_df["solution_value"] = opt_g_df["variable_object"].apply(lambda item: item.varValue)

opt_d_e_df = pd.DataFrame.from_dict(yd_e, orient="index", columns = ["variable_object"])
opt_d_w_df = pd.DataFrame.from_dict(yd_w, orient="index", columns = ["variable_object"])
opt_d_df = pd.concat([opt_d_e_df, opt_d_w_df], ignore_index = True)
opt_d_df["solution_value"] = opt_d_df["variable_object"].apply(lambda item: item.varValue)

In [89]:
opt_g_df

,variable_object,solution_value
0,yg_e_1,50.0
1,yg_e_2,400.0
2,yg_e_3,30.0
3,yg_e_4,0.0
4,yg_e_5,0.0
5,yg_e_6,0.0
6,yg_e_7,0.0
7,yg_w_1,120.0
8,yg_w_2,200.0
9,yg_w_3,60.0


In [90]:
opt_d_df

,variable_object,solution_value
0,yd_e_1,300.0
1,yd_e_2,80.0
2,yd_e_3,60.0
3,yd_e_4,0.0
4,yd_e_5,0.0
5,yd_e_6,0.0
6,yd_w_1,250.0
7,yd_w_2,120.0
8,yd_w_3,40.0
9,yd_w_4,70.0


In [91]:
for name, c in list(opt_model.constraints.items()):
    print(c.pi)

37.5
34.0


In [92]:
print(theta.varValue)

40.0
